# Finetuning d'un LLM à la façon du Conte de Monte Christo

L'objectif de ce projet et de s'entrainer à faire du finetunig de LLM. Pour cela on utilisera la méthode du QLoRA afin de ne pas avoir à entrainer totalement un LLM 'de zéro'.

## Nettoyage du texte : 

Avant toute chose il faut supprimer les pages inutiles (Titre, sommaire, notes, etc...).

## Découpage du texte en segments :

Afin de pouvoir traiter le texte le LLM aura besoin d'un texte découpé en 'chunks' de 4000 à 8000 tokens avec un 'overlap' de 200 tokens.

In [1]:
import os

def prepare_all_chunks(folder_path, chunk_size=8000, overlap=500):
    all_chunks = []
    # On récupère les fichiers et on les trie pour garder l'ordre chronologique (Tome 1 à 4)
    files = sorted([f for f in os.listdir(folder_path) if f.endswith('.txt')])
    
    for file_name in files:
        file_path = os.path.join(folder_path, file_name)
        print(f"Traitement de {file_name}...")
        
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        start = 0
        while start < len(text):
            end = start + chunk_size
            chunk = text[start:end]
            # On ajoute une petite métadonnée pour le suivi
            all_chunks.append({
                "source": file_name,
                "content": chunk
            })
            start += chunk_size - overlap
            
    return all_chunks

chunks_to_process = prepare_all_chunks("media")
print(f"\nTotal de segments à traiter : {len(chunks_to_process)}")

Traitement de rawText_1.txt...
Traitement de rawText_2.txt...
Traitement de rawText_3.txt...
Traitement de rawText_4.txt...

Total de segments à traiter : 374


In [ ]:
import os
from wsgiref import types
from dotenv import load_dotenv
from google import genai

# Necessite l'installation du SDK Google GenAI : python pip install google-genai

# Chargement des variables d'environnement
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("La clé API Gemini n'est pas définie dans le fichier .env")


# Configuration du client
client = genai.Client(api_key=api_key)

SYSTEM_PROMPT = """Tu es un expert en littérature française. Ta mission est d'extraire les répliques du personnage du Comte de Monte-Cristo.
Pour chaque réplique trouvée dans le texte :
1. Crée un champ "instruction" qui contient la phrase ou l'action qui précède la réplique (le contexte).
2. Crée un champ "output" qui contient la réplique exacte du Comte (retire les incises comme "dit-il").
3. Conserve rigoureusement le style d'époque (vouvoiement, vocabulaire).

Renvoie uniquement une LISTE d'objets JSON."""

def test_extraction_gemini(chunks, num_samples=5):
    print(f"Lancement du test Gemini sur {num_samples} segments...")
    
    with open("test_dataset_gemini.jsonl", "w", encoding="utf-8") as f:
        for i in range(num_samples):
            print(f"Traitement du segment {i+1}...")
            
            response = client.models.generate_content(
                model="gemini-2.0-flash", # Vous pouvez aussi utiliser "gemini-1.5-flash"
                config=types.GenerateContentConfig(
                    system_instruction=SYSTEM_PROMPT,
                    response_mime_type="application/json", # Force le format JSON
                    temperature=0
                ),
                contents=chunks[i]['content']
            )
            
            # Gemini renvoie souvent une chaîne JSON représentant une liste
            # On la nettoie pour l'écrire en format JSONL
            try:
                data = response.parsed # Si le SDK supporte le parsing auto
                if not data:
                    import json
                    data = json.loads(response.text)
                
                for entry in data:
                    f.write(json.dumps(entry, ensure_ascii=False) + "\n")
            except Exception as e:
                print(f"Erreur de parsing sur le segment {i} : {e}")

    print("\nTest terminé ! Vérifiez 'test_dataset_gemini.jsonl'.")

# Exécution du test
test_extraction_gemini(chunks_to_process)